# Exploring Data (lesson 1)

In [97]:
#importing packages
from __future__ import print_function
import os
import re
import string
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from numpy import array
from pickle import dump
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


Starting of by downloading data and looking at the data, which we need to clean. We observe following
- Illustration texts
- The text file contain multiple stories, these should be split into multiple files in order for the model to know there is groupings (Extension: will be reviewed in lesson 5)
- The file contains a number of punctuations (,;.* etc.), which are irrelevant for the model
- Capital letters as headers

The input sequences are at first at 50 across sentences, chapters and stories. 

In [98]:
# creating function to load documents into memory
def load_doc(filename):
	file = open(filename, 'r', encoding="utf8") #loading an existing file
	text = file.read() #opening the file and assigning it to the variable text
	file.close() #close the file
	return text #output = text

In [99]:
#concatenate text files
path = 'Data'
 
files = os.listdir(path)

filenames = ['Data/22693-0.txt', 'Data/2892-0.txt', 'Data/32571-0.txt', 'Data/677-0.txt', 'Data/7439-0.txt', 'Data/pg128.txt', 'Data/pg17860.txt', 'Data/pg19068.txt', 'Data/pg19860.txt', 'Data/pg29021.txt','Data/pg4357.txt']

with open('Data/adventures.txt', 'w', encoding="utf8") as outfile:
    for fname in filenames:
        with open(fname, encoding="utf8") as infile:
            outfile.write(infile.read())

In [100]:
# load document
in_filename = 'Data/adventures.txt'#specifying the filename of the data we wish to load
doc = load_doc(in_filename) #loading the file
print(doc[:10000]) #printing the first 200 characters of the loading document

﻿Just as a little child holds out its hands to catch the sunbeams, to
feel and to grasp what, so its eyes tell it, is actually there, so,
down through the ages, men have stretched out their hands in eager
endeavour to know their God. And because only through the human was
the divine knowable, the old peoples of the earth made gods of their
heroes and not unfrequently endowed these gods with as many of the
vices as of the virtues of their worshippers. As we read the myths of
the East and the West we find ever the same story. That portion of the
ancient Aryan race which poured from the central plain of Asia,
through the rocky defiles of what we now call "The Frontier," to
populate the fertile lowlands of India, had gods who must once have
been wholly heroic, but who came in time to be more degraded than the
most vicious of lustful criminals. And the Greeks, Latins, Teutons,
Celts, and Slavonians, who came of the same mighty Aryan stock, did
even as those with whom they owned a common anc

# Cleaning data (lesson 2)

In lesson 1, we observed several potential issues in the data, which we need to remove from the data. This is done below:

In [101]:
#removing illustration descriptions
doc = re.sub(r'\[[^)]*\]', '', doc) #using re.sub function and regular expressions 
#[ - an opening bracket 
#[^()]* - zero or more characters other than those defined, that is, any characters other than [ and ]
#\] - a closing bracket


In [102]:
#removing headers
doc = re.sub(r'[A-Z]{2,}','', doc) #replacing capital letters longer than 1 with nothing. 


In [103]:
#removing special characters, since one of the deliminators
#doc = doc.replace("'", '')

#deliminators 
#deliminators = c('THE FIR TREE', 'LITTLE TUK', 'THE UGLY DUCKLING', 'LITTLE IDA'S FLOWERS', 'THE STEADFAST TIN SOLDIER, LITTLE THUMBELINA, SUNSHINE STORIES', 'THE DARNING-NEEDLE', 'THE LITTLE MATCH GIRL', 'THE LOVING PAIR', 'THE LEAPING MATCH', 'THE HAPPY FAMILY, THE GREENIES, 'OLE-LUK-OIE', 'THE DREAM GOD', 'THE MONEY BOX', 'ELDER-TREE MOTHER', 'THE SNOW QUEEN', 'THE ROSES AND THE SPARROWS', 'THE OLD HOUSE', 'THE CONCEITED APPLE BRANCH' 

#splitting the file into strings consisting of one adventure
#doc = doc.split ()

In [104]:
# turn a doc into clean tokens
def clean_doc(doc): #making a function
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token - https://www.geeksforgeeks.org/python-maketrans-translate-functions/
	table = str.maketrans('', '', string.punctuation) #Third argument specifies the wished deleted items
	tokens = [w.translate(table) for w in tokens] #translate applies the translation table applied on the looping through the tokens list
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()] #Replace word with word, if the word is alphabetic
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [105]:
tokens = clean_doc(doc) #using the clean function to cleaning our document
print(tokens[:200])
print('Total Tokens: %d' % len(tokens)) #printing the number of tokens
print('Unique Tokens: %d' % len(set(tokens))) #printing the number of unique tokens by grouping similair tokens

['as', 'a', 'little', 'child', 'holds', 'out', 'its', 'hands', 'to', 'catch', 'the', 'sunbeams', 'to', 'feel', 'and', 'to', 'grasp', 'what', 'so', 'its', 'eyes', 'tell', 'it', 'is', 'actually', 'there', 'so', 'down', 'through', 'the', 'ages', 'men', 'have', 'stretched', 'out', 'their', 'hands', 'in', 'eager', 'endeavour', 'to', 'know', 'their', 'god', 'and', 'because', 'only', 'through', 'the', 'human', 'was', 'the', 'divine', 'knowable', 'the', 'old', 'peoples', 'of', 'the', 'earth', 'made', 'gods', 'of', 'their', 'heroes', 'and', 'not', 'unfrequently', 'endowed', 'these', 'gods', 'with', 'as', 'many', 'of', 'the', 'vices', 'as', 'of', 'the', 'virtues', 'of', 'their', 'worshippers', 'as', 'we', 'read', 'the', 'myths', 'of', 'the', 'east', 'and', 'the', 'west', 'we', 'find', 'ever', 'the', 'same', 'story', 'that', 'portion', 'of', 'the', 'ancient', 'aryan', 'race', 'which', 'poured', 'from', 'the', 'central', 'plain', 'of', 'asia', 'through', 'the', 'rocky', 'defiles', 'of', 'what', 'w

In [106]:
# organize into sequences of tokens
length = 50 + 1 #defining the length of the sequence
sequences = list() #creating an empty list
for i in range(length, len(tokens)): #range takes two arguments: start point and end point
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq) #-join returns a string in which the string elements of sequence have been joined by str separator
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 423208


In [107]:
# save tokens to file, one dialog per line
def save_doc(lines, filename): #creating a saving function
	data = '\n'.join(lines) #joining the lines seperated by 'enter'
	file = open(filename, 'w', encoding="utf8") #create empty new file
	file.write(data) #input the data in the empty file
	file.close() #close the file

In [108]:
# save sequences to file
out_filename = 'Data/fairytales_sequences.txt'
save_doc(sequences, out_filename)

# Training the model (Lesson 3)

In [109]:
# loading the cleaned data
in_filename = 'Data/fairytales_sequences.txt'
doc = load_doc(in_filename)

#split the text by lineshifts creating a list of 51 items long string
lines = doc.split('\n')

In [110]:
# integer encode sequences of words
tokenizer = Tokenizer() #assign the tokenizer function to a variable
tokenizer.fit_on_texts(lines) #function of keras finds all of the unique words in the data and assigns each a unique integer
sequences = tokenizer.texts_to_sequences(lines) #translating the input lines into integers
tokenizer.word_index #checking the dictionary of the transformed wordsb

{'the': 1,
 'and': 2,
 'to': 3,
 'of': 4,
 'a': 5,
 'he': 6,
 'in': 7,
 'that': 8,
 'i': 9,
 'was': 10,
 'his': 11,
 'it': 12,
 'you': 13,
 'her': 14,
 'she': 15,
 'with': 16,
 'for': 17,
 'had': 18,
 'as': 19,
 'but': 20,
 'they': 21,
 'him': 22,
 'said': 23,
 'at': 24,
 'not': 25,
 'on': 26,
 'my': 27,
 'so': 28,
 'all': 29,
 'is': 30,
 'me': 31,
 'be': 32,
 'when': 33,
 'then': 34,
 'have': 35,
 'this': 36,
 'by': 37,
 'them': 38,
 'one': 39,
 'were': 40,
 'from': 41,
 'which': 42,
 'who': 43,
 'will': 44,
 'out': 45,
 'their': 46,
 'up': 47,
 'there': 48,
 'into': 49,
 'no': 50,
 'came': 51,
 'what': 52,
 'went': 53,
 'your': 54,
 'we': 55,
 'would': 56,
 'king': 57,
 'if': 58,
 'little': 59,
 'are': 60,
 'man': 61,
 'could': 62,
 'do': 63,
 'down': 64,
 'an': 65,
 'great': 66,
 'time': 67,
 'did': 68,
 'upon': 69,
 'before': 70,
 'where': 71,
 'more': 72,
 'or': 73,
 'very': 74,
 'day': 75,
 'some': 76,
 'after': 77,
 'see': 78,
 'been': 79,
 'come': 80,
 'now': 81,
 'go': 82,
 's

In [111]:
#calculating vocabulary size to estimate the size of the embedding layer
vocab_size = len(tokenizer.word_index) + 1 #since indexing of array are zero-offset, the index of the vocabulary must be one larger than the length
vocab_size

16259

In [112]:
# separate into input and output
sequences = array(sequences) #transforming the sequens of integers to arrays
X, y = sequences[:,:-1], sequences[:,-1] #defining X (input sequences) and y (output words)
y = to_categorical(y, num_classes=vocab_size) #to_categorical converts a class vector (integers) to binary class matrix
seq_length = X.shape[1] #gives you the dimension of the array, which we put to be 50
seq_length

50

We will use a two LSTM hidden layers with 100 memory cells each. More memory cells and a deeper network may achieve better results.

In [113]:
# define model
model = Sequential() #assigning the sequential function to a model
model.add(Embedding(vocab_size, 50, input_length=seq_length)) #defining embedding layer size
model.add(LSTM(100, return_sequences=True)) #adding layer of nodes
model.add(LSTM(100))  #adding layer of nodes
model.add(Dense(100, activation='relu')) #specifying the structure of the hidden layer, recu is an argument of a rectified linear unit. 
model.add(Dense(vocab_size, activation='softmax')) #using the softmax function to creating probabilities
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            812950    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 16259)             1642159   
Total params: 2,606,009
Trainable params: 2,606,009
Non-trainable params: 0
_________________________________________________________________
None


In [114]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #the compile function configures the model for training specifying the categorical cross entropy loss
# fit model
model.fit(X, y, batch_size=128, epochs=100) #training the model 

Epoch 1/100
342016/423208 [=======================>......] - ETA: 5:40 - loss: 6.4049 - acc: 0.0733

ResourceExhaustedError: OOM when allocating tensor with shape[16259,50] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[Node: training/Adam/Square = Square[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](training/Adam/mul_2/y)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

# Lesson 4

In [ ]:
# load cleaned text sequences
doc = load_doc('Data/fairytales_sequences.txt')
lines = doc.split('\n')

In [ ]:
#Specifying input sequences length to prompt the model
seq_length = len(lines[3].split()) - 1 #splitting the sequences into words, counting them -1
seq_length

In [ ]:
#loading model
model= load_model('model.h5')
tokenizer = load(open('tokenizer.pkl', 'rb')) #r for read

In [ ]:
#selecting a seed text
seed_text = lines[randint(0,len(lines))] #returns random integer between 0 and how many lines there is, and indexes this. 
print(seed_text + '\n') #prints the selected text
len(seed_text.split())

In [ ]:
#translating the input text to integers
encoded = tokenizer.texts_to_sequences([seed_text])[0]
encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')

The model can predict the next word directly by calling model.predict_classes() that will return the index of the word with the highest probability

In [ ]:
# predict probabilities for each word
yhat = model.predict_classes(encoded, verbose=1)

Looking up the index in the Tokenizers mapping to get the associated word

In [ ]:
#making a loop to translate integer to word
out_word = ''
for word, index in tokenizer.word_index.items():
	if index == yhat:
		out_word = word
		break

out_word

The input sequences will get too long, in order to keep them to 50 items using the following function, which pads sequences to the same lengt

In [ ]:
#Setting the max length to be 50 items by removing items from the beginnning of the sequence
encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')

In [ ]:
#creating a function, which generates the predicted output
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list() #make an empty list
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [ ]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

#saving generated text
out_filename = 'Data/first_output.txt'
save_doc(generated, out_filename)


Things to be consider to advance the model: 

- We could process the data so that the model only ever deals with self-contained sentences and pad or truncate the text to meet this requirement for each input sequence. You could explore this as an extension to this tutorial.
- MORE DATA!

